In [1]:
import sys
sys.path.insert(0, '../../')

In [3]:
import backtrader as bt
import pandas as pd
import numpy as np
import datetime
from copy import deepcopy
from backtrader.data_reader import get_fund_nav_data, get_all_fund_codes, get_date_range, test_connection

In [4]:
def format_nav_data(df):
    df_formatted = df.rename(columns={'open_price': 'open',
                                      'close_price': 'close',
                                      'high_price': 'high',
                                      'low_price': 'low',
                                      'nav_date': 'datetime'})
    
    df_formatted = df_formatted[['datetime','fund_code', 'open', 'high', 'low', 'close', 'volume']].copy()
    df_formatted['volume'] = df_formatted['volume'].astype(int)
    df_formatted['open'] = df_formatted['open'].astype(float)
    df_formatted['high'] = df_formatted['high'].astype(float)
    df_formatted['low'] = df_formatted['low'].astype(float)
    df_formatted['close'] = df_formatted['close'].astype(float)
    df_formatted['openinterest'] = 0
    df_formatted['datetime'] = pd.to_datetime(df_formatted['datetime'])
    return df_formatted

# 一、读取日度行情表

表内字段就是 Backtrader 默认情况下要求输入的 7 个字段： 'datetime' 、'open'、'high'、'low'、'close'、'volume'、'openinterest'，外加一个 'sec_code' 股票代码字段。

In [5]:
daily_price = get_fund_nav_data(fund_codes=['513310','518880'],
                              start_date='2024-01-01', end_date='2026-01-01')
daily_price = format_nav_data(daily_price)
daily_price.head()

,datetime,fund_code,open,high,low,close,volume,openinterest
0,2024-01-02,513310,1.235,1.237,1.224,1.228,130393,0
1,2024-01-03,513310,1.216,1.216,1.193,1.193,152362,0
2,2024-01-04,513310,1.188,1.194,1.177,1.183,105238,0
3,2024-01-05,513310,1.185,1.188,1.165,1.169,248912,0
4,2024-01-08,513310,1.173,1.174,1.147,1.148,108920,0


In [6]:
# 筛选 513310 和 518880 2只基金的数据集
data1 = daily_price.query(f"fund_code=='513310'").set_index('datetime').drop(columns=['fund_code'])
data2 = daily_price.query(f"fund_code=='518880'").set_index('datetime').drop(columns=['fund_code'])
data2.head()

,open,high,low,close,volume,openinterest
datetime,,,,,,
2024-01-02,4.643,4.667,4.643,4.666,2062784,0
2024-01-03,4.658,4.666,4.655,4.663,1111962,0
2024-01-04,4.640,4.656,4.633,4.651,1149663,0
2024-01-05,4.659,4.665,4.656,4.664,1267497,0
2024-01-08,4.663,4.667,4.646,4.650,1161639,0


# 一、 简单移动均线的测试案例

策略：5 日均线上穿 10 日均线 → 买入；5 日均线下穿 10 日均线 → 卖出

### 注：后面各交易操作都基于该案例进行测试

In [ ]:
class TestStrategy(bt.Strategy):
  
    params=(('period1',10),
            ('period2',20),)   #全局设定均线周期
    
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
    
    def __init__(self):
        #计算均线
        self.ma1 = bt.indicators.SMA(self.data0, period=self.p.period1)
        self.ma2 = bt.indicators.SMA(self.data0, period=self.p.period2)
        #计算2条均线交叉信号：ma2 上穿 ma1 时，取值为 +1； ma2 下穿 ma1 时，取值为 -1
        #self.crossover = bt.indicators.CrossOver(self.ma2, self.ma1) 
        # 初始化订单
        self.order = None
        
    def next(self):
        signal = 0
        if abs(self.ma1[0]-self.ma2[0]) <= 0.01:
            signal = 1
        elif (self.ma1[0]-self.ma2[0]) >= 0.25:
            signal = -1
        else:
            signal = 0
        # 打印当前时刻的总资产
        self.log('当前总资产 %.2f' %(self.broker.getvalue()))
        # 取消之前未执行的订单
        if self.order:  
            self.cancel(self.order)  
        
        # 检查是否有持仓
        if not self.position:
            # 10日均线上穿5日均线，买入
            if signal > 0 and self.broker.getcash()>=10000:             
                self.order = self.buy(size=100*(10000//(self.data0.close[0]*100))) # 以下一日开盘价买入100股
                #self.order = self.buy(size=10000)
        # # 10日均线下穿5日均线，卖出
        elif signal < 0:            
            self.order = self.close(size=10000) # 平仓，以下一日开盘价卖出
            
            
    
    def notify_order(self, order):
        # 未被处理的订单
        if order.status in [order.Submitted, order.Accepted]:
            return
        # 已被处理的订单
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, ref:%.0f，Price: %.4f, Size: %.2f, Cost: %.4f, Comm %.4f, Stock: %s' %
                    (order.ref,
                     order.executed.price,
                     order.executed.size,
                     order.executed.value,
                     order.executed.comm,
                     order.data._name))
            else:  # Sell
                self.log('SELL EXECUTED, ref:%.0f, Price: %.4f, Size: %.2f, Cost: %.4f, Comm %.4f, Stock: %s' %
                        (order.ref,
                        order.executed.price,
                         order.executed.size,
                        order.executed.value,
                        order.executed.comm,
                        order.data._name))
                
        
# 实例化大脑
cerebro1= bt.Cerebro()
# 设置初始资金
cerebro1.broker.set_cash(100000)
# 加载数据
datafeed1 = bt.feeds.PandasData(dataname=data2, fromdate=datetime.datetime(2025,1,1), todate=datetime.datetime(2026,1,1))
cerebro1.adddata(datafeed1, name='518880')
# 添加策略
cerebro1.addstrategy(TestStrategy) 
# 添加分析器
cerebro1.addanalyzer(bt.analyzers.TimeReturn, _name='pnl') # 返回收益率时序数据
cerebro1.addanalyzer(bt.analyzers.AnnualReturn, _name='_AnnualReturn')
cerebro1.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.003, annualize=True, _name='_SharpeRatio')
cerebro1.addanalyzer(bt.analyzers.DrawDown, _name='_DrawDown')
# 添加观测器
cerebro1.addobserver(bt.observers.Value)  # 查看账户资产变动 
# 添加观测器 
cerebro1.addobserver(bt.observers.DrawDown)
cerebro1.addobserver(bt.observers.TimeReturn)
cerebro1.addobserver(bt.observers.Benchmark, data=datafeed1)         
# 启动回测
result = cerebro1.run()  

2025-02-06, 当前总资产 100000.00
2025-02-07, 当前总资产 100000.00
2025-02-10, 当前总资产 100000.00
2025-02-11, 当前总资产 100000.00
2025-02-12, 当前总资产 100000.00
2025-02-13, 当前总资产 100000.00
2025-02-14, 当前总资产 100000.00
2025-02-17, 当前总资产 100000.00
2025-02-18, 当前总资产 100000.00
2025-02-19, 当前总资产 100000.00
2025-02-20, 当前总资产 100000.00
2025-02-21, 当前总资产 100000.00
2025-02-24, 当前总资产 100000.00
2025-02-25, 当前总资产 100000.00
2025-02-26, 当前总资产 100000.00
2025-02-27, 当前总资产 100000.00
2025-02-28, 当前总资产 100000.00
2025-03-03, 当前总资产 100000.00
2025-03-04, 当前总资产 100000.00
2025-03-05, 当前总资产 100000.00
2025-03-06, BUY EXECUTED, ref:265，Price: 6.5300, Size: 1500.00, Cost: 9795.0000, Comm 0.0000, Stock: 518880
2025-03-06, 当前总资产 100007.50
2025-03-07, 当前总资产 99994.00
2025-03-10, 当前总资产 100021.00
2025-03-11, 当前总资产 99962.50
2025-03-12, 当前总资产 99994.00
2025-03-13, 当前总资产 100067.50
2025-03-14, 当前总资产 100214.50
2025-03-17, 当前总资产 100220.50
2025-03-18, 当前总资产 100295.50
2025-03-19, 当前总资产 100405.00
2025-03-20, 当前总资产 100415.50
2025-03-21, 当前总资产 100366.00

In [25]:
strat = result[0]
print("--------------- AnnualReturn -----------------")
print(strat.analyzers._AnnualReturn.get_analysis())
print("--------------- SharpeRatio -----------------")
print(strat.analyzers._SharpeRatio.get_analysis())
print("--------------- DrawDown -----------------")
print(strat.analyzers._DrawDown.get_analysis())

--------------- AnnualReturn -----------------
OrderedDict([(2025, -0.028835000000000055)])
--------------- SharpeRatio -----------------
OrderedDict([('sharperatio', None)])
--------------- DrawDown -----------------
AutoOrderedDict([('len', 55), ('drawdown', 8.635360857232902), ('moneydown', 9179.0), ('max', AutoOrderedDict([('len', 90), ('drawdown', 11.426165736084782), ('moneydown', 12145.5)]))])


In [26]:
cerebro1.plot(iplot=False)

[[<Figure size 1920x981 with 8 Axes>]]

# 二、 打印资金和持仓

In [32]:
class TestStrategy(bt.Strategy):
  
    params=(('period1',5),
            ('period2',20),)   #全局设定均线周期
    
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
    
    def __init__(self):
        #计算均线
        self.ma1 = bt.indicators.SMA(self.data0, period=self.p.period1)
        self.ma2 = bt.indicators.SMA(self.data0, period=self.p.period2)
        #计算2条均线交叉信号：ma2 上穿 ma1 时，取值为 +1； ma2 下穿 ma1 时，取值为 -1
        self.crossover = bt.indicators.CrossOver(self.ma2, self.ma1) 
        # 初始化订单
        self.order = None
        
    def next(self):
        
        # 打印每日的资金和持仓情况
        print('date', self.data0.datetime.date(0))
        print('当前可用资金', self.broker.getcash())
        print('当前总资产', self.broker.getvalue())
        print('当前持仓量', self.broker.getposition(self.data).size)
        print('当前持仓成本', self.broker.getposition(self.data).price)
        
        # 取消之前未执行的订单
        if self.order:  
            self.cancel(self.order)  
        
        # 检查是否有持仓
        if not self.position or self.broker.getcash()>=10000:  
            # 10日均线上穿5日均线，买入
            if self.crossover > 0:             
                self.order = self.buy(size=100*int(10000//(self.data0.close[0]*100))) # 以下一日开盘价买入100股
        # # 10日均线下穿5日均线，卖出
        elif self.crossover < 0:            
            self.order = self.close(size=self.position.size//2) # 平仓，以下一日开盘价卖出
            
    
    def notify_order(self, order):
        # 未被处理的订单
        if order.status in [order.Submitted, order.Accepted]:
            return
        # 已被处理的订单
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, ref:%.0f，Price: %.4f, Size: %.2f, Cost: %.4f, Comm %.4f, Stock: %s' %
                    (order.ref,
                     order.executed.price,
                     order.executed.size,
                     order.executed.value,
                     order.executed.comm,
                     order.data._name))
            else:  # Sell
                self.log('SELL EXECUTED, ref:%.0f, Price: %.4f, Size: %.2f, Cost: %.4f, Comm %.4f, Stock: %s' %
                        (order.ref,
                        order.executed.price,
                         order.executed.size,
                        order.executed.value,
                        order.executed.comm,
                        order.data._name))
                
# 实例化大脑
cerebro1= bt.Cerebro()
# 设置初始资金
cerebro1.broker.set_cash(100000)
# 加载数据
datafeed1 = bt.feeds.PandasData(dataname=data2, fromdate=datetime.datetime(2024,1,1), todate=datetime.datetime(2026,1,1))
cerebro1.adddata(datafeed1, name='518880')
# 添加策略
cerebro1.addstrategy(TestStrategy)       
# 添加观测器
cerebro1.addobserver(bt.observers.Value)  # 查看账户资产变动 
cerebro1.addobserver(bt.observers.DrawDown)
cerebro1.addobserver(bt.observers.TimeReturn)
cerebro1.addobserver(bt.observers.Benchmark, data=datafeed1)          
# 启动回测
result = cerebro1.run()  

date 2024-01-30
当前可用资金 100000.0
当前总资产 100000.0
当前持仓量 0
当前持仓成本 0.0
date 2024-01-31
当前可用资金 100000.0
当前总资产 100000.0
当前持仓量 0
当前持仓成本 0.0
date 2024-02-01
当前可用资金 100000.0
当前总资产 100000.0
当前持仓量 0
当前持仓成本 0.0
date 2024-02-02
当前可用资金 100000.0
当前总资产 100000.0
当前持仓量 0
当前持仓成本 0.0
date 2024-02-05
当前可用资金 100000.0
当前总资产 100000.0
当前持仓量 0
当前持仓成本 0.0
date 2024-02-06
当前可用资金 100000.0
当前总资产 100000.0
当前持仓量 0
当前持仓成本 0.0
date 2024-02-07
当前可用资金 100000.0
当前总资产 100000.0
当前持仓量 0
当前持仓成本 0.0
date 2024-02-08
当前可用资金 100000.0
当前总资产 100000.0
当前持仓量 0
当前持仓成本 0.0
date 2024-02-19
当前可用资金 100000.0
当前总资产 100000.0
当前持仓量 0
当前持仓成本 0.0
date 2024-02-20
当前可用资金 100000.0
当前总资产 100000.0
当前持仓量 0
当前持仓成本 0.0
date 2024-02-21
当前可用资金 100000.0
当前总资产 100000.0
当前持仓量 0
当前持仓成本 0.0
date 2024-02-22
当前可用资金 100000.0
当前总资产 100000.0
当前持仓量 0
当前持仓成本 0.0
date 2024-02-23
当前可用资金 100000.0
当前总资产 100000.0
当前持仓量 0
当前持仓成本 0.0
2024-02-26, BUY EXECUTED, ref:325，Price: 4.6470, Size: 2100.00, Cost: 9758.7000, Comm 0.0000, Stock: 518880
date 2024-02-26
当前可用资金 90241.3
当前总

In [33]:
cerebro1.plot(iplot=False)

[[<Figure size 1920x981 with 9 Axes>]]

# 三、 滑点管理

In [38]:
class TestStrategy(bt.Strategy):
  
    params=(('period1',5),
            ('period2',10),)   #全局设定均线周期
    
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
    
    def __init__(self):
        #计算均线
        self.ma1 = bt.indicators.SMA(self.data0, period=self.p.period1)
        self.ma2 = bt.indicators.SMA(self.data0, period=self.p.period2)
        #计算2条均线交叉信号：ma2 上穿 ma1 时，取值为 +1； ma2 下穿 ma1 时，取值为 -1
        self.crossover = bt.indicators.CrossOver(self.ma2, self.ma1) 
        # 初始化订单
        self.order = None
        
    def next(self):
        
        # 打印每日的资金和持仓情况
        print('date', self.data0.datetime.date(0),
              'close', self.data0.close[0],
              'open', self.data0.open[0])
        print('当前可用资金', self.broker.getcash())
        print('当前总资产', self.broker.getvalue())
        print('当前持仓量', self.broker.getposition(self.data).size)
        print('当前持仓成本', self.broker.getposition(self.data).price)
        
        # 取消之前未执行的订单
        if self.order:  
            self.cancel(self.order)  
        
        # 检查是否有持仓
        if not self.position or self.broker.getcash()>=10000:  
            # 10日均线上穿5日均线，买入
            if self.crossover > 0:             
                self.order = self.buy(size=100*int(10000//(self.data0.close[0]*100))) # 以下一日开盘价买入100股
        # # 10日均线下穿5日均线，卖出
        elif self.crossover < 0:            
            self.order = self.close(size=self.position.size//2) # 平仓，以下一日开盘价卖出
            
    
    def notify_order(self, order):
        # 未被处理的订单
        if order.status in [order.Submitted, order.Accepted]:
            return
        # 已被处理的订单
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, ref:%.0f，Price: %.4f, Size: %.2f, Cost: %.4f, Comm %.4f, Stock: %s' %
                    (order.ref,
                     order.executed.price,
                     order.executed.size,
                     order.executed.value,
                     order.executed.comm,
                     order.data._name))
            else:  # Sell
                self.log('SELL EXECUTED, ref:%.0f, Price: %.4f, Size: %.2f, Cost: %.4f, Comm %.4f, Stock: %s' %
                        (order.ref,
                        order.executed.price,
                         order.executed.size,
                        order.executed.value,
                        order.executed.comm,
                        order.data._name))
                
# 实例化大脑
cerebro1= bt.Cerebro()
# 设置初始资金
cerebro1.broker.set_cash(100000)
# 加载数据
datafeed1 = bt.feeds.PandasData(dataname=data1, fromdate=datetime.datetime(2025,1,1), todate=datetime.datetime(2026,1,1))
cerebro1.adddata(datafeed1, name='513310')


# 设置百分比滑点 
cerebro1.broker.set_slippage_perc(perc=0.001) 
# 设置固定滑点
# cerebro1.broker.set_slippage_fixed(fixed=0.001) 
# 有关滑点的其他设置
# cerebro1.broker.set_slippage_fixed(fixed=0.35, slip_open=False, slip_match=True, slip_out=False)
# cerebro1.broker.set_slippage_fixed(fixed=0.35, slip_open=True, slip_match=True, slip_out=False)
# cerebro1.broker.set_slippage_fixed(fixed=0.35, slip_open=True, slip_match=True, slip_out=True)
# cerebro1.broker.set_slippage_fixed(fixed=0.35, slip_open=True, slip_match=False, slip_out=True)

# 添加策略
cerebro1.addstrategy(TestStrategy)            
# 启动回测
result = cerebro1.run()  

date 2025-01-16 close 1.495 open 1.519
当前可用资金 100000.0
当前总资产 100000.0
当前持仓量 0
当前持仓成本 0.0
date 2025-01-17 close 1.521 open 1.495
当前可用资金 100000.0
当前总资产 100000.0
当前持仓量 0
当前持仓成本 0.0
date 2025-01-20 close 1.523 open 1.535
当前可用资金 100000.0
当前总资产 100000.0
当前持仓量 0
当前持仓成本 0.0
date 2025-01-21 close 1.541 open 1.533
当前可用资金 100000.0
当前总资产 100000.0
当前持仓量 0
当前持仓成本 0.0
date 2025-01-22 close 1.564 open 1.545
当前可用资金 100000.0
当前总资产 100000.0
当前持仓量 0
当前持仓成本 0.0
date 2025-01-23 close 1.53 open 1.559
当前可用资金 100000.0
当前总资产 100000.0
当前持仓量 0
当前持仓成本 0.0
date 2025-01-24 close 1.536 open 1.541
当前可用资金 100000.0
当前总资产 100000.0
当前持仓量 0
当前持仓成本 0.0
date 2025-01-27 close 1.494 open 1.533
当前可用资金 100000.0
当前总资产 100000.0
当前持仓量 0
当前持仓成本 0.0
date 2025-02-05 close 1.487 open 1.508
当前可用资金 100000.0
当前总资产 100000.0
当前持仓量 0
当前持仓成本 0.0
date 2025-02-06 close 1.522 open 1.485
当前可用资金 100000.0
当前总资产 100000.0
当前持仓量 0
当前持仓成本 0.0
2025-02-07, BUY EXECUTED, ref:356，Price: 1.5225, Size: 6500.00, Cost: 9896.3865, Comm 0.0000, Stock: 513310
dat

In [ ]:
# 2019-01-17: 开盘价成交，实际成交价 为 开盘价*（1+0.001）
32.63307367 * (1+0.001)

32.66570674367

In [39]:
cerebro1.plot(iplot=False)

[[<Figure size 1920x981 with 5 Axes>]]

# 四、交易税费管理

以 考虑佣金和印花税的股票百分比费用 为例

In [40]:
# 考虑佣金和印花税的股票百分比费用
class StockCommission(bt.CommInfoBase):
    params = (('stamp_duty', 0.001),
              ('stocklike', True),  # 指定为期货模式
              ('commtype', bt.CommInfoBase.COMM_PERC),  # 使用百分比费用模式
              ('percabs', True),)  # commission 不以 % 为单位 # 印花税默认为 0.1%
    
    def _getcommission(self,size,price,pseudoexec):
        if size>0: # 买入时，只考虑佣金
            return abs(size) * price * self.p.commission
        elif size<0: # 卖出时，同时考虑佣金和印花税
            return abs(size) * price * (self.p.commission + self.p.stamp_duty)
        else:
            return 0
  

                
# 实例化大脑
cerebro1= bt.Cerebro()
# 设置初始资金
cerebro1.broker.set_cash(100000)
# 加载数据
datafeed1 = bt.feeds.PandasData(dataname=data1, fromdate=datetime.datetime(2025,1,1), todate=datetime.datetime(2026,1,1))
cerebro1.adddata(datafeed1, name='513310')

# 设置交易费用
comminfo = StockCommission(commission=0.0002, stamp_duty=0.001) # 实例化 
cerebro1.broker.addcommissioninfo(comminfo)

# 添加策略
cerebro1.addstrategy(TestStrategy)            
# 启动回测
result = cerebro1.run()  

date 2025-01-16 close 1.495 open 1.519
当前可用资金 100000.0
当前总资产 100000.0
当前持仓量 0
当前持仓成本 0.0
date 2025-01-17 close 1.521 open 1.495
当前可用资金 100000.0
当前总资产 100000.0
当前持仓量 0
当前持仓成本 0.0
date 2025-01-20 close 1.523 open 1.535
当前可用资金 100000.0
当前总资产 100000.0
当前持仓量 0
当前持仓成本 0.0
date 2025-01-21 close 1.541 open 1.533
当前可用资金 100000.0
当前总资产 100000.0
当前持仓量 0
当前持仓成本 0.0
date 2025-01-22 close 1.564 open 1.545
当前可用资金 100000.0
当前总资产 100000.0
当前持仓量 0
当前持仓成本 0.0
date 2025-01-23 close 1.53 open 1.559
当前可用资金 100000.0
当前总资产 100000.0
当前持仓量 0
当前持仓成本 0.0
date 2025-01-24 close 1.536 open 1.541
当前可用资金 100000.0
当前总资产 100000.0
当前持仓量 0
当前持仓成本 0.0
date 2025-01-27 close 1.494 open 1.533
当前可用资金 100000.0
当前总资产 100000.0
当前持仓量 0
当前持仓成本 0.0
date 2025-02-05 close 1.487 open 1.508
当前可用资金 100000.0
当前总资产 100000.0
当前持仓量 0
当前持仓成本 0.0
date 2025-02-06 close 1.522 open 1.485
当前可用资金 100000.0
当前总资产 100000.0
当前持仓量 0
当前持仓成本 0.0
2025-02-07, BUY EXECUTED, ref:369，Price: 1.5210, Size: 6500.00, Cost: 9886.5000, Comm 1.9773, Stock: 513310
dat

In [41]:
# 2019-01-17 买入时只考虑 佣金： Cost * commission
100 * 32.6331 * 0.0002

0.6526620000000001

In [42]:
#  2019-01-29 卖出时 考虑佣金和印花税 
100 * 33.9285 * (0.0002+0.001)

4.071420000000001

# 五、成交量限制管理

In [44]:
# 将 原始数据的 volume 缩小 10000 倍，方便模拟结果
data1_ = data1.copy()
data1_.loc[:,'volume'] = (data1_.loc[:,'volume'] / 10000).astype('int')

In [50]:
data1_.head()

,open,high,low,close,volume,openinterest
datetime,,,,,,
2024-01-02,1.235,1.237,1.224,1.228,13,0
2024-01-03,1.216,1.216,1.193,1.193,15,0
2024-01-04,1.188,1.194,1.177,1.183,10,0
2024-01-05,1.185,1.188,1.165,1.169,24,0
2024-01-08,1.173,1.174,1.147,1.148,10,0


## bt.fillers.FixedSize(size)

In [51]:
class TestStrategy(bt.Strategy):
  
    params=(('period1',5),
            ('period2',10),)   #全局设定均线周期
    
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
    
    def __init__(self):
        #计算均线
        self.ma1 = bt.indicators.SMA(self.data0, period=self.p.period1)
        self.ma2 = bt.indicators.SMA(self.data0, period=self.p.period2)
        #计算2条均线交叉信号：ma2 上穿 ma1 时，取值为 +1； ma2 下穿 ma1 时，取值为 -1
        self.crossover = bt.indicators.CrossOver(self.ma2, self.ma1) 
        # 初始化订单
        self.order = None
        
    def next(self):
         # 打印日志
       
        print('date', self.datas[0].datetime.date(0),
              'open', self.data0.open[0],
              'volume', self.data0.volume[0],
              '当前持仓量', self.broker.getposition(self.data).size,
              '当前持仓成本', self.broker.getposition(self.data).price)
    
        # 取消之前未执行的订单
        if self.order:  
            self.cancel(self.order)  
        
        # 检查是否有持仓
        if not self.position:  
            # 10日均线上穿5日均线，买入
            if self.crossover > 0:             
                self.order = self.buy(size=2000) # 以下一日开盘价买入100股
        # # 10日均线下穿5日均线，卖出
        elif self.crossover < 0:            
            self.order = self.close() # 平仓，以下一日开盘价卖出
            
    
    def notify_order(self, order):
        # 未被处理的订单
        if order.status in [order.Created, order.Submitted, order.Accepted, order.Canceled, order.Margin]:
            return
        # 已被处理的订单
        if order.status in [order.Partial, order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, ref:%.0f，Price: %.4f, Size: %.2f, Remsize: %.2f, Cost: %.4f, Stock: %s' %
                    (order.ref,
                     order.executed.price,
                     order.executed.size,
                     order.executed.remsize, # 剩余 数量
                     order.executed.value,
                     order.data._name))
            else:  # Sell
                self.log('SELL EXECUTED, ref:%.0f, Price: %.4f, Size: %.2f, Remsize: %.2f, Cost: %.4f, Stock: %s' %
                        (order.ref,
                        order.executed.price,
                         order.executed.size,
                         order.executed.remsize, # 剩余 数量
                        order.executed.value,
                        order.data._name))
                
# 实例化大脑
cerebro4 = bt.Cerebro()
# 设置初始资金
cerebro4.broker.set_cash(100000)
# 设置成交量限制
cerebro4.broker.set_filler(bt.broker.fillers.FixedSize(size=3000))
# cerebro6.broker.set_filler(bt.broker.fillers.BarPointPerc(minmov=0.1, perc=0.5))
# 加载数据
datafeed1 = bt.feeds.PandasData(dataname=data1_, fromdate=datetime.datetime(2025,1,2), todate=datetime.datetime(2026,1,1))
cerebro4.adddata(datafeed1, name='513310')
# 添加策略
cerebro4.addstrategy(TestStrategy)
# 添加观测器
cerebro4.addobserver(bt.observers.Value)  # 查看账户资产变动 
cerebro4.addobserver(bt.observers.DrawDown)
cerebro4.addobserver(bt.observers.TimeReturn)
cerebro4.addobserver(bt.observers.Benchmark, data=datafeed1)             
# 启动回测
result = cerebro4.run()  

date 2025-01-16 open 1.519 volume 615.0 当前持仓量 0 当前持仓成本 0.0
date 2025-01-17 open 1.495 volume 1369.0 当前持仓量 0 当前持仓成本 0.0
date 2025-01-20 open 1.535 volume 495.0 当前持仓量 0 当前持仓成本 0.0
date 2025-01-21 open 1.533 volume 581.0 当前持仓量 0 当前持仓成本 0.0
date 2025-01-22 open 1.545 volume 521.0 当前持仓量 0 当前持仓成本 0.0
date 2025-01-23 open 1.559 volume 681.0 当前持仓量 0 当前持仓成本 0.0
date 2025-01-24 open 1.541 volume 436.0 当前持仓量 0 当前持仓成本 0.0
date 2025-01-27 open 1.533 volume 345.0 当前持仓量 0 当前持仓成本 0.0
date 2025-02-05 open 1.508 volume 447.0 当前持仓量 0 当前持仓成本 0.0
date 2025-02-06 open 1.485 volume 615.0 当前持仓量 0 当前持仓成本 0.0
2025-02-07, BUY EXECUTED, ref:382，Price: 1.5210, Size: 544.00, Remsize: 1456.00, Cost: 827.4240, Stock: 513310
date 2025-02-07 open 1.521 volume 544.0 当前持仓量 544.0 当前持仓成本 1.521
date 2025-02-10 open 1.523 volume 410.0 当前持仓量 544.0 当前持仓成本 1.521
date 2025-02-11 open 1.55 volume 340.0 当前持仓量 544.0 当前持仓成本 1.521
date 2025-02-12 open 1.534 volume 450.0 当前持仓量 544.0 当前持仓成本 1.521
2025-02-13, SELL EXECUTED, ref:383, Pri

In [ ]:
# 可视化观测结果
cerebro4.plot(iplot=False)

## bt.broker.fillers.FixedBarPerc(perc)

In [101]:
class TestStrategy(bt.Strategy):
  
    params=(('period1',5),
            ('period2',10),)   #全局设定均线周期
    
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
    
    def __init__(self):
        #计算均线
        self.ma1 = bt.indicators.SMA(self.data0, period=self.p.period1)
        self.ma2 = bt.indicators.SMA(self.data0, period=self.p.period2)
        #计算2条均线交叉信号：ma2 上穿 ma1 时，取值为 +1； ma2 下穿 ma1 时，取值为 -1
        self.crossover = bt.indicators.CrossOver(self.ma2, self.ma1) 
        # 初始化订单
        self.order = None
        
    def next(self):
         # 打印日志
       
        print('date', self.datas[0].datetime.date(0),
              'open', self.data0.open[0],
              'volume', self.data0.volume[0],
              '当前持仓量', self.broker.getposition(self.data).size,
              '当前持仓成本', self.broker.getposition(self.data).price)
    
        # 取消之前未执行的订单
        if self.order:  
            self.cancel(self.order)  
        
        # 检查是否有持仓
        if not self.position:  
            # 10日均线上穿5日均线，买入
            if self.crossover > 0:             
                self.order = self.buy(size=2000) # 以下一日开盘价买入2000股
        # # 10日均线下穿5日均线，卖出
        elif self.crossover < 0:            
            self.order = self.close() # 平仓，以下一日开盘价卖出
            
    
    def notify_order(self, order):
        # 未被处理的订单
        if order.status in [order.Created, order.Submitted, order.Accepted, order.Canceled, order.Margin]:
            return
        # 已被处理的订单
        if order.status in [order.Partial, order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, ref:%.0f，Price: %.4f, Size: %.2f, Remsize: %.2f, Cost: %.4f, Stock: %s' %
                    (order.ref,
                     order.executed.price,
                     order.executed.size,
                     order.executed.remsize, # 剩余 数量
                     order.executed.value,
                     order.data._name))
            else:  # Sell
                self.log('SELL EXECUTED, ref:%.0f, Price: %.4f, Size: %.2f, Remsize: %.2f, Cost: %.4f, Stock: %s' %
                        (order.ref,
                        order.executed.price,
                         order.executed.size,
                         order.executed.remsize, # 剩余 数量
                        order.executed.value,
                        order.data._name))
                
# 实例化大脑
cerebro1 = bt.Cerebro()
# 设置初始资金
cerebro1.broker.set_cash(1000000)
# 设置成交量限制
cerebro1.broker.set_filler(bt.broker.fillers.FixedBarPerc(perc=50))  # 表示 50% 
# cerebro6.broker.set_filler(bt.broker.fillers.BarPointPerc(minmov=0.1, perc=0.5))
# 加载数据
datafeed1 = bt.feeds.PandasData(dataname=data1_, fromdate=datetime.datetime(2019,1,2), todate=datetime.datetime(2021,1,28))
cerebro1.adddata(datafeed1, name='600466.SH')
# 添加策略
cerebro1.addstrategy(TestStrategy)            
# 启动回测
result = cerebro1.run()  

date 2019-01-16 open 33.00320305 volume 660.0 当前持仓量 0 当前持仓成本 0.0
2019-01-17, BUY EXECUTED, ref:2664，Price: 32.6331, Size: 434.00, Remsize: 1566.00, Cost: 14162.7540, Stock: 600466.SH
date 2019-01-17 open 32.63307367 volume 869.0 当前持仓量 434.0 当前持仓成本 32.63307367
date 2019-01-18 open 31.95450314 volume 890.0 当前持仓量 434.0 当前持仓成本 32.63307367
date 2019-01-21 open 33.06489128 volume 1292.0 当前持仓量 434.0 当前持仓成本 32.63307367
date 2019-01-22 open 32.38632075 volume 1030.0 当前持仓量 434.0 当前持仓成本 32.63307367
date 2019-01-23 open 32.44800898 volume 421.0 当前持仓量 434.0 当前持仓成本 32.63307367
date 2019-01-24 open 32.50969721 volume 615.0 当前持仓量 434.0 当前持仓成本 32.63307367
date 2019-01-25 open 32.94151482 volume 1732.0 当前持仓量 434.0 当前持仓成本 32.63307367
date 2019-01-28 open 33.311644199999996 volume 1686.0 当前持仓量 434.0 当前持仓成本 32.63307367
2019-01-29, SELL EXECUTED, ref:2665, Price: 33.9285, Size: -434.00, Remsize: 0.00, Cost: 14162.7540, Stock: 600466.SH
date 2019-01-29 open 33.928526500000004 volume 2256.0 当前持仓量 0.0 当前持仓成本 0

## bt.broker.fillers.BarPointPerc(minmov=0.01，perc=100.0)

In [99]:
class TestStrategy(bt.Strategy):
  
    params=(('period1',5),
            ('period2',10),)   #全局设定均线周期
    
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
    
    def __init__(self):
        #计算均线
        self.ma1 = bt.indicators.SMA(self.data0, period=self.p.period1)
        self.ma2 = bt.indicators.SMA(self.data0, period=self.p.period2)
        #计算2条均线交叉信号：ma2 上穿 ma1 时，取值为 +1； ma2 下穿 ma1 时，取值为 -1
        self.crossover = bt.indicators.CrossOver(self.ma2, self.ma1) 
        # 初始化订单
        self.order = None
        
    def next(self):
         # 打印日志
       
        print('date', self.datas[0].datetime.date(0),
              'open', self.data0.open[0],
              'high', self.data0.high[0],
              'low', self.data0.low[0],
              'volume', self.data0.volume[0],
              '当前持仓量', self.broker.getposition(self.data).size,
              '当前持仓成本', self.broker.getposition(self.data).price)
    
        # 取消之前未执行的订单
        if self.order:  
            self.cancel(self.order)  
        
        # 检查是否有持仓
        if not self.position:  
            # 10日均线上穿5日均线，买入
            if self.crossover > 0:             
                self.order = self.buy(size=1000) # 以下一日开盘价买入100股
        # # 10日均线下穿5日均线，卖出
        elif self.crossover < 0:            
            self.order = self.close() # 平仓，以下一日开盘价卖出
            
    
    def notify_order(self, order):
        # 未被处理的订单
        if order.status in [order.Created, order.Submitted, order.Accepted, order.Canceled, order.Margin]:
            return
        # 已被处理的订单
        if order.status in [order.Partial, order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, ref:%.0f，Price: %.4f, Size: %.2f, Remsize: %.2f, Cost: %.4f, Stock: %s' %
                    (order.ref,
                     order.executed.price,
                     order.executed.size,
                     order.executed.remsize, # 剩余 数量
                     order.executed.value,
                     order.data._name))
            else:  # Sell
                self.log('SELL EXECUTED, ref:%.0f, Price: %.4f, Size: %.2f, Remsize: %.2f, Cost: %.4f, Stock: %s' %
                        (order.ref,
                        order.executed.price,
                        order.executed.size,
                        order.executed.remsize, # 剩余 数量
                        order.executed.value,
                        order.data._name))
                
# 实例化大脑
cerebro7 = bt.Cerebro()
# 设置初始资金
cerebro7.broker.set_cash(1000000)
# 设置成交量限制
cerebro7.broker.set_filler(bt.broker.fillers.BarPointPerc(minmov=0.1, perc=50))
# 加载数据
datafeed1 = bt.feeds.PandasData(dataname=data1_, fromdate=datetime.datetime(2019,1,2), todate=datetime.datetime(2021,1,28))
cerebro7.adddata(datafeed1, name='600466.SH')
# 添加策略
cerebro7.addstrategy(TestStrategy)            
# 启动回测
result = cerebro7.run()  

date 2019-01-16 open 33.00320305 high 33.00320305 low 32.57138544 volume 660.0 当前持仓量 0 当前持仓成本 0.0
2019-01-17, BUY EXECUTED, ref:2560，Price: 32.6331, Size: 36.00, Remsize: 964.00, Cost: 1174.7907, Stock: 600466.SH
date 2019-01-17 open 32.63307367 high 32.94151482 low 31.83112668 volume 869.0 当前持仓量 36.0 当前持仓成本 32.63307367
date 2019-01-18 open 31.95450314 high 32.81813836 low 31.95450314 volume 890.0 当前持仓量 36.0 当前持仓成本 32.63307367
date 2019-01-21 open 33.06489128 high 33.06489128 low 32.0778796 volume 1292.0 当前持仓量 36.0 当前持仓成本 32.63307367
date 2019-01-22 open 32.38632075 high 32.6947619 low 32.01619137 volume 1030.0 当前持仓量 36.0 当前持仓成本 32.63307367
date 2019-01-23 open 32.44800898 high 32.57138544 low 32.0778796 volume 421.0 当前持仓量 36.0 当前持仓成本 32.63307367
date 2019-01-24 open 32.50969721 high 32.57138544 low 31.89281491 volume 615.0 当前持仓量 36.0 当前持仓成本 32.63307367
date 2019-01-25 open 32.94151482 high 34.05190296 low 32.75645013 volume 1732.0 当前持仓量 36.0 当前持仓成本 32.63307367
date 2019-01-28 open 33.

In [82]:
parts = (32.94151482 - 31.83112668 + 0.1)  // 0.1 
parts

12.0

In [86]:
(869.0 / parts) * 50 / 100

36.208333333333336

# 交易时机的管理

## 正常模式：当日收盘后下单，次日以开盘价成交

In [120]:
class TestStrategy(bt.Strategy):
  
    params=(('period1',5),
            ('period2',10),)   #全局设定均线周期
    
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
    
    def __init__(self):
        #计算均线
        self.ma1 = bt.indicators.SMA(self.data0, period=self.p.period1)
        self.ma2 = bt.indicators.SMA(self.data0, period=self.p.period2)
        #计算2条均线交叉信号：ma2 上穿 ma1 时，取值为 +1； ma2 下穿 ma1 时，取值为 -1
        self.crossover = bt.indicators.CrossOver(self.ma2, self.ma1) 
        # 初始化订单
        self.order = None
        
    def next(self):
        # 取消之前未执行的订单
        if self.order:  
            self.cancel(self.order)  
        # 检查是否有持仓
        if not self.position:  
            # 10日均线上穿5日均线，买入
            if self.crossover > 0:  
                print('{} Send Buy, open {}'.format(self.data.datetime.date(),self.data.open[0]))
                self.order = self.buy(size=100) # 以下一日开盘价买入100股
        # # 10日均线下穿5日均线，卖出
        elif self.crossover < 0:   
            print('{} Send Sell, open {}'.format(self.data.datetime.date(),self.data.open[0]))
            self.order = self.close() # 平仓，以下一日开盘价卖出
    
    def notify_order(self, order):
        if order.status != order.Completed:
            return
        self.order = None
        print('{} {} Executed at price {}'.format(
            bt.num2date(order.executed.dt).date(),
            'Buy' * order.isbuy() or 'Sell', order.executed.price))        

    
# 实例化大脑
cerebro5= bt.Cerebro(cheat_on_open=False)
# 设置初始资金
cerebro5.broker.set_cash(1000000000000)
# 加载数据
datafeed1 = bt.feeds.PandasData(dataname=data1, fromdate=datetime.datetime(2019,1,2), todate=datetime.datetime(2021,1,28))
cerebro5.adddata(datafeed1, name='600466.SH')
# 添加策略
cerebro5.addstrategy(TestStrategy)            
# 启动回测
result = cerebro5.run()  

2019-01-16 Send Buy, open 33.00320305
2019-01-17 Buy Executed at price 32.63307367
2019-01-28 Send Sell, open 33.311644199999996
2019-01-29 Sell Executed at price 33.928526500000004
2019-02-21 Send Buy, open 35.47073225
2019-02-22 Buy Executed at price 34.91553818
2019-02-26 Send Sell, open 37.07462623
2019-02-27 Sell Executed at price 37.50644384
2019-03-14 Send Buy, open 41.82461994
2019-03-15 Buy Executed at price 41.20773764
2019-03-15 Send Sell, open 41.20773764
2019-03-18 Sell Executed at price 44.10708445
2019-03-29 Send Buy, open 43.55189038
2019-04-01 Buy Executed at price 46.14279604
2019-04-02 Send Sell, open 48.1168194
2019-04-03 Sell Executed at price 47.06811949
2019-04-16 Send Buy, open 47.56162533
2019-04-17 Buy Executed at price 46.63630188
2019-05-17 Send Sell, open 40.93009102
2019-05-20 Sell Executed at price 39.77351074
2019-05-20 Send Buy, open 39.77351074
2019-05-21 Buy Executed at price 39.25947506
2019-06-12 Send Sell, open 39.90201966
2019-06-13 Sell Executed 

## cheat_on_open

In [121]:
class TestStrategy(bt.Strategy):
  
    params=(('period1',5),
            ('period2',10),)   #全局设定均线周期
    
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
    
    def __init__(self):
        #计算均线
        self.ma1 = bt.indicators.SMA(self.data0, period=self.p.period1)
        self.ma2 = bt.indicators.SMA(self.data0, period=self.p.period2)
        #计算2条均线交叉信号：ma2 上穿 ma1 时，取值为 +1； ma2 下穿 ma1 时，取值为 -1
        self.crossover = bt.indicators.CrossOver(self.ma2, self.ma1) 
        # 初始化订单
        self.order = None
        
    def next_open(self):
        # 取消之前未执行的订单
        if self.order:  
            self.cancel(self.order)  
        # 检查是否有持仓
        if not self.position:  
            # 10日均线上穿5日均线，买入
            if self.crossover > 0:  
                print('{} Send Buy, open {}'.format(self.data.datetime.date(),self.data.open[0]))
                self.order = self.buy(size=100) # 以下一日开盘价买入100股
        # # 10日均线下穿5日均线，卖出
        elif self.crossover < 0:  
            print('{} Send Sell, open {}'.format(self.data.datetime.date(),self.data.open[0]))
            self.order = self.close() # 平仓，以下一日开盘价卖出
    
    def notify_order(self, order):
        if order.status != order.Completed:
            return
        self.order = None
        print('{} {} Executed at price {}'.format(
            bt.num2date(order.executed.dt).date(),
            'Buy' * order.isbuy() or 'Sell', order.executed.price))        

    
# 实例化大脑
cerebro5= bt.Cerebro(cheat_on_open=True)
# 设置初始资金
cerebro5.broker.set_cash(1000000000000)
# 加载数据
datafeed1 = bt.feeds.PandasData(dataname=data1, fromdate=datetime.datetime(2019,1,2), todate=datetime.datetime(2021,1,28))
cerebro5.adddata(datafeed1, name='600466.SH')
# 添加策略
cerebro5.addstrategy(TestStrategy)            
# 启动回测
result = cerebro5.run()  

2019-01-17 Send Buy, open 32.63307367
2019-01-17 Buy Executed at price 32.63307367
2019-01-29 Send Sell, open 33.928526500000004
2019-01-29 Sell Executed at price 33.928526500000004
2019-02-22 Send Buy, open 34.91553818
2019-02-22 Buy Executed at price 34.91553818
2019-02-27 Send Sell, open 37.50644384
2019-02-27 Sell Executed at price 37.50644384
2019-03-15 Send Buy, open 41.20773764
2019-03-15 Buy Executed at price 41.20773764
2019-03-18 Send Sell, open 44.10708445
2019-03-18 Sell Executed at price 44.10708445
2019-04-01 Send Buy, open 46.14279604
2019-04-01 Buy Executed at price 46.14279604
2019-04-03 Send Sell, open 47.06811949
2019-04-03 Sell Executed at price 47.06811949
2019-04-17 Send Buy, open 46.63630188
2019-04-17 Buy Executed at price 46.63630188
2019-05-20 Send Sell, open 39.77351074
2019-05-20 Sell Executed at price 39.77351074
2019-05-21 Send Buy, open 39.25947506
2019-05-21 Buy Executed at price 39.25947506
2019-06-13 Send Sell, open 40.09478304
2019-06-13 Sell Executed

## Cheat-On-Close

In [122]:
class TestStrategy(bt.Strategy):
  
    params=(('period1',5),
            ('period2',10),)   #全局设定均线周期
    
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
    
    def __init__(self):
        #计算均线
        self.ma1 = bt.indicators.SMA(self.data0, period=self.p.period1)
        self.ma2 = bt.indicators.SMA(self.data0, period=self.p.period2)
        #计算2条均线交叉信号：ma2 上穿 ma1 时，取值为 +1； ma2 下穿 ma1 时，取值为 -1
        self.crossover = bt.indicators.CrossOver(self.ma2, self.ma1) 
        # 初始化订单
        self.order = None
        
    def next(self):
        # 取消之前未执行的订单
        if self.order:  
            self.cancel(self.order)  
        
        # 检查是否有持仓
        if not self.position:  
            # 10日均线上穿5日均线，买入
            if self.crossover > 0:  
                print('{} Send Buy, close {}'.format(self.data.datetime.date(),self.data.close[0]))
                self.order = self.buy(size=100) # 以下一日开盘价买入100股
        # # 10日均线下穿5日均线，卖出
        elif self.crossover < 0:
            print('{} Send Sell, close {}'.format(self.data.datetime.date(),self.data.close[0]))
            self.order = self.close() # 平仓，以下一日开盘价卖出
    
    def notify_order(self, order):
        if order.status != order.Completed:
            return
        self.order = None
        print('{} {} Executed at price {}'.format(
            bt.num2date(order.executed.dt).date(),
            'Buy' * order.isbuy() or 'Sell', order.executed.price))        

    
# 实例化大脑
cerebro5= bt.Cerebro()
# 设置初始资金
cerebro5.broker.set_cash(1000000000000)
# 当日下单，当日收盘价成交
cerebro5.broker.set_coc(True)
# 加载数据
datafeed1 = bt.feeds.PandasData(dataname=data1, fromdate=datetime.datetime(2019,1,2), todate=datetime.datetime(2021,1,28))
cerebro5.adddata(datafeed1, name='600466.SH')
# 添加策略
cerebro5.addstrategy(TestStrategy)            
# 启动回测
result = cerebro5.run()  

2019-01-16 Send Buy, close 32.63307367
2019-01-16 Buy Executed at price 32.63307367
2019-01-28 Send Sell, close 33.86683827
2019-01-28 Sell Executed at price 33.86683827
2019-02-21 Send Buy, close 34.85384995
2019-02-21 Buy Executed at price 34.85384995
2019-02-26 Send Sell, close 37.75319676
2019-02-26 Sell Executed at price 37.75319676
2019-03-14 Send Buy, close 41.20773764
2019-03-14 Buy Executed at price 41.20773764
2019-03-15 Send Sell, close 42.62656693
2019-03-15 Sell Executed at price 42.626566929999996
2019-03-29 Send Buy, close 45.52591374
2019-03-29 Buy Executed at price 45.52591373999999
2019-04-02 Send Sell, close 47.43824887
2019-04-02 Sell Executed at price 47.43824887000001
2019-04-16 Send Buy, close 46.63630188
2019-04-16 Buy Executed at price 46.63630188
2019-05-17 Send Sell, close 39.96627412
2019-05-17 Sell Executed at price 39.96627412
2019-05-20 Send Buy, close 39.38798398
2019-05-20 Buy Executed at price 39.38798398
2019-06-12 Send Sell, close 39.96627412
2019-06